## DSC Courses: Assignment Grades Main

In [1]:
import sys
sys.path.insert(0, 'src')

# tables
import numpy as np
import pandas as pd

# basic packages
from glob import glob
import json
import re

# grades, canvas
from canvasapi import Canvas
from third_parties import *

### Setup

#### Canvas Setup

In [2]:
# load credential
CREDENTIAL_FILE = '.info/canvas_credentials.json'
with open(CREDENTIAL_FILE) as f:
    canvas_credential = json.load(f)

In [3]:
# Setup: Canvas
API_URL = "https://canvas.ucsd.edu/"
API_KEY = canvas_credential['API_KEY']        # Generate Access Token from Canvas
COURSE_ID = canvas_credential['COURSE_ID']    # From Canvas Website
COURSE = Canvas(API_URL, API_KEY).get_course(COURSE_ID)

print(COURSE.name) # make sure title and quarter is correct

DSC 20 - Prgrmng/DataStruc for Data Sci - Langlois [SP24]


#### Personnel Setup

In [4]:
# Setup: Student Profiles (ID, Name, Email)
student_ls = []
for stu in COURSE.get_users(enrollment_type=['student'], enrollment_state=['active']):
    student_ls.append([stu.id, stu.name, stu.email])
STUDENTS = pd.DataFrame(student_ls, columns=["id", "name", "email"])
STUDENTS.to_csv(".info/students.csv")
print(STUDENTS.shape[0])

145


In [5]:
# Setup: Staff Profiles
STAFF = pd.read_csv('.info/staff_wi24.csv')['Email'].unique()

In [6]:
# Setup: Mismatched emails
with open('.info/email_records.json') as f:
    EMAIL_RECORDS = json.load(f)
EMAIL_RECORDS

{'devonchang918@gmail.com': 'd6chang@ucsd.edu',
 'honghang3623@gmail.com': 'ax009357@acsmail.ucsd.edu'}

### Slido

In [ ]:
# setup
fname = "lec01.xlsx"  # change file name
aname = 'Lecture 01 (09/23)'  # change for each lecture

# create gradebook
slido_lec = Slido(COURSE, STUDENTS, STAFF, EMAIL_RECORDS, fname, aname)
gradebook = slido_lec.create_gradebook()
gradebook

In [ ]:
# once inspected, enter grades
slido_lec.enter_grades()

### WebClicker

In [19]:
# discussions
# setup
fname = "DSC20_SP24 Votes.csv"  # change file name
aname = 'Discussion 10 (06/05)'  # change for each lecture

# create gradebook
wc = WebClicker(COURSE, STUDENTS, STAFF, EMAIL_RECORDS, fname, aname)
gradebook = wc.create_gradebook(session=41, min_poll=0.5) # change session name
gradebook

,id,name,typed_email,Last Name,First Name,Student Identifier,Session 41,Unnamed: 147,Unnamed: 148,Unnamed: 149,Unnamed: 150,Discussion 10 (06/05)
email,,,,,,,,,,,,
aaa004@ucsd.edu,129436.0,Aadit Agarwal,aaa004@ucsd.edu,Agarwal,Aadit,A16997098,NaN,C,A,B,B,1.0
ragrawal@ucsd.edu,166096.0,Reva Agrawal,ragrawal@ucsd.edu,Agrawal,Reva,NaN,NaN,NaN,NaN,NaN,A,0.0
maahluwalia@ucsd.edu,188881.0,Maanav Ahluwalia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
makyildiz@ucsd.edu,106805.0,Melisa Akyildiz,makyildiz@ucsd.edu,Akyildiz,Melisa,A16645913,NaN,NaN,NaN,NaN,NaN,0.0
dbahiru@ucsd.edu,194127.0,Daniel Bahiru,dbahiru@ucsd.edu,Bahiru,Daniel,A17745117,NaN,NaN,NaN,D,A,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
sih005@ucsd.edu,NaN,NaN,sih005@ucsd.edu,Huang,Sirui,A16634129,NaN,NaN,NaN,NaN,NaN,0.0
sjatia@ucsd.edu,NaN,NaN,sjatia@ucsd.edu,Jatia,Siya,A17353222,NaN,NaN,NaN,NaN,NaN,0.0
tchima@ucsd.edu,NaN,NaN,tchima@ucsd.edu,Chima,Taranvir,A18136894,NaN,NaN,NaN,NaN,NaN,0.0


In [20]:
# once inspected, enter grades
wc.enter_grades()

Processing Participation: Discussion 10 (06/05)
Updating Canvas Grades - Main Assignment
done



,id,name,typed_email,Last Name,First Name,Student Identifier,Session 41,Unnamed: 147,Unnamed: 148,Unnamed: 149,Unnamed: 150,Discussion 10 (06/05)
email,,,,,,,,,,,,
abharris@ucsd.edu,NaN,NaN,abharris@ucsd.edu,Harris,Andrew,A16193456,NaN,NaN,NaN,NaN,NaN,0.0
alw033@ucsd.edu,NaN,NaN,alw033@ucsd.edu,Wang,Alice,A17989632,NaN,NaN,NaN,NaN,NaN,0.0
asg010@ucsd.edu,NaN,NaN,asg010@ucsd.edu,Gupta,Ashutosh,A17752840,NaN,NaN,NaN,NaN,NaN,0.0
b5zhang@ucsd.edu,NaN,NaN,b5zhang@ucsd.edu,Zhang,Beichen,A16636515,NaN,NaN,NaN,NaN,NaN,0.0
cac031@ucsd.edu,NaN,NaN,cac031@ucsd.edu,Chen,Cameron,A18125167,NaN,NaN,NaN,NaN,NaN,0.0
d3ding@ucsd.edu,NaN,NaN,d3ding@ucsd.edu,Ding,Ding,A17089007,NaN,NaN,NaN,NaN,NaN,0.0
d6chang@ucsd.edu,NaN,NaN,devonchang918@gmail.com,Chang,Devin,A16970346,NaN,NaN,NaN,NaN,NaN,0.0
dparameshwaran@ucsd.edu,NaN,NaN,dparameshwaran@ucsd.edu,Parameshwaran,Diya,A17287963,NaN,NaN,NaN,NaN,NaN,0.0
hol050@ucsd.edu,NaN,NaN,hol050@ucsd.edu,LIM,HO,A18071915,NaN,NaN,NaN,NaN,NaN,0.0


### Zybook Readings

In [ ]:
# setup
fname = "lec01.csv"  # change file name
aname = 'Zybook Readings 01 (09/23)'  # change for each assignment
due_time = None

# create gradebook
zy_reading = Zybook(COURSE, STUDENTS, STAFF, EMAIL_RECORDS, fname, aname, due_time=due_time)
config = {
    '10.1': ['Participation', 'Challenge'],
    '10.2': ['Participation', 'Challenge'],
    '10.3': ['Participation', 'Challenge']
}
gradebook = zy_reading.create_gradebook(config=config)
gradebook

In [ ]:
# once inspected, enter grades
zy_reading.enter_grades()

### Discussion Zylabs

In [ ]:
# setup
fname = "disc01.csv"  # change file name
aname = 'Discussion Zylab Week 1 (09/27)'  # change for each assignment
due_time = '2022-11-22 23:59'

# create gradebook
disc_zylab = Zybook(
    COURSE, STUDENTS, STAFF, EMAIL_RECORDS, fname, aname,
    dir_name='zylab',
    assignment_group='Discussion Zylab',
    due_time=due_time
)
config = {
    '9.12': ['Lab'],
    '9.13': ['Lab'],
    '13.4': ['Participation'],
    '13.8': ['Lab'],
    '26.21': ['Lab']
}
gradebook = disc_zylab.create_gradebook(config=config)
gradebook

In [ ]:
# once inspected, enter grades
disc_zylab.enter_grades()

### Gradescope Homework/Project

In [ ]:
aname = 'hw01'  # change for each assignment
fname = aname + "_scores.csv"  # main file name
due_time = '2022-01-17 23:59'

# other files
lateness_file = aname + "_lateness.csv"  # lateness file name
redemption_file = aname + "_redemption.csv" # redemption file name
other_section_files = {
    'checkpoint': aname + '_checkpoint.csv',
    # 'quiz': aname + '_quiz.csv'
}

# create gradebook
gs_hw = Gradescope(
    COURSE, STUDENTS, STAFF, EMAIL_RECORDS, fname, aname,
    lateness_policy='slip_day',
    lateness_file=lateness_file,
    redemption_file=redemption_file,
    redemption_rate=0.8,
    total_slip_days=5,
    other_section_files=other_section_files,
    due_time=due_time
)

# late_policy = {
#     'no late': 1.0,
#     '1 day': 0.8,
#     '2 days': 0.5
# }

gradebook = gs_hw.create_gradebook()  # defaults take 1 slip day
gradebook

In [ ]:
# once inspected, enter grades
mismatch = gs_hw.enter_grades()
mismatch

In [ ]:
gs_hw.process_slip_day()

### Student Search

In [ ]:
# find student
STUDENTS[STUDENTS['email'].str.lower().str.contains('<email>')]

In [ ]:
# find grade: version 1
gradebook[gradebook.index.str.lower().str.contains('<email>', na=False)]

In [ ]:
# find grade: version 2
gradebook[gradebook['name'].str.lower().str.contains('<name>', na=False)]

### Debug Results

In [30]:
STUDENTS[STUDENTS['name'] == 'Devin Chang']

,id,name,email
13,130215,Devin Chang,d6chang@ucsd.edu


In [31]:
# modify EMAIL_RECORDS
EMAIL_RECORDS['devonchang918@gmail.com'] = 'd6chang@ucsd.edu'

In [32]:
EMAIL_RECORDS

{'honghang3623@gmail.com': 'ax009357@acsmail.ucsd.edu',
 'devonchang918@gmail.com': 'd6chang@ucsd.edu'}

In [33]:
# DO NOT FORGET to update file
with open('.info/email_records.json', 'w') as f:
    json.dump(EMAIL_RECORDS, f, sort_keys=True)